In [42]:
import numpy as np
from itertools import combinations
import networkx as nx
from collections import defaultdict
import os



def generate_possible_pairs(p, e, b):
    polish_only = list(range(1, p + 1))
    english_only = list(range(p + 1, p + e + 1))
    bilingual = list(range(p + e + 1, p + e + b + 1))

    possible_pairs = []

    # Polish-only pairs
    possible_pairs.extend(combinations(polish_only, 2))
    # English-only pairs
    possible_pairs.extend(combinations(english_only, 2))
    # Polish and bilingual
    possible_pairs.extend([(po, bi) for po in polish_only for bi in bilingual])
    # English and bilingual
    possible_pairs.extend([(eo, bi) for eo in english_only for bi in bilingual])
    # Bilingual pairs
    possible_pairs.extend(combinations(bilingual, 2))

    return possible_pairs

def max_rounds(p, e, b):
    possible_pairs = generate_possible_pairs(p, e, b)

    G = nx.Graph()
    G.add_edges_from(possible_pairs)

    rounds = 0
    total_people = p + e + b
    if total_people % 2 != 0:
        return 0, []  # If odd number of people, cannot pair everyone in each round

    all_round_pairs = []
    
    while True:
        matching = nx.max_weight_matching(G, maxcardinality=True)
        if len(matching) * 2 != total_people:
            break  # If we can't pair everyone, stop the rounds

        # Convert set of pairs to a sorted list of tuples for consistent output
        round_pairs = sorted(list(matching))
        all_round_pairs.append(round_pairs)
        print(f"Round {rounds + 1}: {round_pairs}")

        # Remove the matched pairs from the graph
        G.remove_edges_from(matching)
        
        rounds += 1

    return rounds, all_round_pairs

def assign_tables_to_rounds(round_pairs):
    table_assignments = []
    n_tables = len(round_pairs[0])  # Determine number of tables from the first round
    
    for round_num, pairs in enumerate(round_pairs, start=1):
        table_assignment = {}
        for table_number, pair in enumerate(pairs, start=1):
            table_assignment[pair] = table_number
        table_assignments.append(table_assignment)
    return table_assignments

def generate_visible_ids(p, e, b):
    polish_only = [f"P{i:03}" for i in range(1, p + 1)]
    english_only = [f"E{i:03}" for i in range(1, e + 1)]
    bilingual = [f"B{i:03}" for i in range(1, b + 1)]
    visible_ids = polish_only+english_only+bilingual

    return {id:visible_id for id, visible_id in zip(range(1,p+e+b+1), visible_ids)}


def generate_seating_guides(round_pairs, table_assignments):
    participant_info = defaultdict(list)
    for round_number, (pairs, table_assignment) in enumerate(zip(round_pairs, table_assignments), start=1):
        for pair in pairs:
            table_number = table_assignment[pair]
            participant_info[pair[0]].append({'round':round_number, 'table':table_number})
            participant_info[pair[1]].append({'round':round_number, 'table':table_number})
    return participant_info

def generate_html_files(participant_info, visible_ids, n_rounds=10):
    os.makedirs('participants', exist_ok=True)

    for participant_id, info_list in participant_info.items():
        with open(f'participants/{visible_ids[participant_id]}.html', 'w') as f:
            f.write(f"<html><body><h1>Participant {visible_ids[participant_id]}</h1>\n")
            for i in range(n_rounds):
                f.write(f"<p>{info_list[i]}</p>\n")
            f.write("</body></html>")





In [43]:
p = 14  # number of polish-only speakers
e = 1  # number of english-only speakers
b = 33  # number of bilingual speakers

rounds, round_pairs = max_rounds(p, e, b)
table_assignments = assign_tables_to_rounds(round_pairs)
participant_info = generate_seating_guides(round_pairs, table_assignments)


# Output participant info
for participant_id, info in participant_info.items():
    print(f"Participant {participant_id}:\n" + "\n".join(info) + "\n")
generate_html_files(participant_info, generate_visible_ids(p,e,b))


Round 1: [(15, 16), (26, 25), (27, 24), (28, 23), (29, 22), (30, 21), (31, 20), (32, 19), (33, 18), (34, 17), (35, 14), (36, 13), (37, 12), (38, 11), (39, 10), (40, 9), (41, 8), (42, 7), (43, 6), (44, 5), (45, 4), (46, 3), (47, 2), (48, 1)]
Round 2: [(15, 17), (26, 24), (27, 25), (28, 22), (29, 23), (30, 20), (31, 21), (32, 18), (33, 19), (34, 16), (35, 13), (36, 14), (37, 11), (38, 12), (39, 9), (40, 10), (41, 7), (42, 8), (43, 5), (44, 6), (45, 3), (46, 4), (47, 1), (48, 2)]
Round 3: [(3, 25), (15, 18), (27, 23), (28, 24), (29, 21), (30, 22), (31, 19), (32, 20), (33, 14), (34, 13), (35, 17), (36, 16), (37, 10), (38, 9), (39, 12), (40, 11), (41, 6), (42, 5), (43, 8), (44, 7), (45, 2), (46, 1), (47, 4), (48, 26)]
Round 4: [(5, 24), (15, 19), (27, 22), (28, 25), (29, 20), (30, 23), (31, 18), (32, 21), (33, 17), (34, 14), (35, 16), (36, 12), (37, 13), (38, 10), (39, 11), (40, 8), (41, 9), (42, 6), (43, 7), (44, 4), (45, 1), (46, 2), (47, 26), (48, 3)]
Round 5: [(15, 20), (25, 23), (27, 2

In [26]:
print(participant_info)

defaultdict(<class 'list'>, {15: ['Round 1: Table 1', 'Round 2: Table 1', 'Round 3: Table 2', 'Round 4: Table 2', 'Round 5: Table 1', 'Round 6: Table 2', 'Round 7: Table 1', 'Round 8: Table 1', 'Round 9: Table 1', 'Round 10: Table 2', 'Round 11: Table 2', 'Round 12: Table 1', 'Round 13: Table 1', 'Round 14: Table 3', 'Round 15: Table 4', 'Round 16: Table 5', 'Round 17: Table 2', 'Round 18: Table 1', 'Round 19: Table 2', 'Round 20: Table 3', 'Round 21: Table 5', 'Round 22: Table 6', 'Round 23: Table 6', 'Round 24: Table 6', 'Round 25: Table 5', 'Round 26: Table 5', 'Round 27: Table 4', 'Round 28: Table 6', 'Round 29: Table 6', 'Round 30: Table 3', 'Round 31: Table 1', 'Round 32: Table 1', 'Round 33: Table 1'], 16: ['Round 1: Table 1', 'Round 2: Table 10', 'Round 3: Table 12', 'Round 4: Table 11', 'Round 5: Table 9', 'Round 6: Table 8', 'Round 7: Table 13', 'Round 8: Table 15', 'Round 9: Table 7', 'Round 10: Table 6', 'Round 11: Table 17', 'Round 12: Table 18', 'Round 13: Table 5', 'Roun